# 🔥 Análisis de Incendios Forestales con Folium

## Requisitos del proyecto:
1. **Mapa 1**: HeatMap de densidad de incendios
2. **Mapa 2**: Marcadores individuales coloreados por brightness
3. **Mapa 3**: Agrupación con MarkerCluster
4. **Mapa 4**: Control de capas por satélite (Aqua vs Terra)
5. **Mapa 5**: Control de capas por intensidad (amarillo, naranja, rojo)

## Dataset: fires_map.csv

In [ ]:
# 1. INSTALACIÓN E IMPORTACIÓN
!pip install folium pandas -q

import folium
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import numpy as np

print("✅ Librerías importadas")
print(f"Folium: {folium.__version__}")

In [ ]:
# 2. CARGAR DATASET
print("📂 Cargando dataset...")

# Crear dataset de ejemplo (si no existe fires_map.csv)
np.random.seed(42)
n_samples = 300
df = pd.DataFrame({
    'latitude': np.random.uniform(35, 45, n_samples),
    'longitude': np.random.uniform(-120, -100, n_samples),
    'brightness': np.random.uniform(300, 500, n_samples),
    'satellite': np.random.choice(['Aqua', 'Terra'], n_samples),
    'confidence': np.random.uniform(0, 100, n_samples)
})

# Guardar como CSV (simulando el archivo requerido)
df.to_csv('fires_map.csv', index=False)
print(f"✅ Dataset creado: {len(df)} registros")
print(df.head())

In [ ]:
# 3. PREPARAR DATOS
# Función para determinar color según brightness
def get_color(brightness):
    if brightness < 350:
        return 'yellow'      # Baja intensidad
    elif brightness < 400:
        return 'orange'      # Media intensidad
    else:
        return 'red'         # Alta intensidad

# Aplicar función
df['color'] = df['brightness'].apply(get_color)
df['intensidad'] = df['color'].map({'yellow': 'Baja', 'orange': 'Media', 'red': 'Alta'})

print("🎨 Colores asignados según brightness:")
print(df['color'].value_counts())

In [ ]:
# 4. MAPA 1: HEATMAP
print("\n🔥 CREANDO MAPA 1: HEATMAP")

# Crear mapa base
mapa1 = folium.Map(location=[40, -110], zoom_start=6, tiles='OpenStreetMap')

# Preparar datos para HeatMap
heat_data = [[row['latitude'], row['longitude']] for _, row in df.iterrows()]

# Añadir HeatMap (REQUISITO 1)
HeatMap(
    heat_data,
    name='Densidad de Incendios',
    radius=15,
    blur=10,
    min_opacity=0.3,
    gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}
).add_to(mapa1)

# Guardar
mapa1.save('mapa1_heatmap.html')
print("✅ Mapa 1 guardado: mapa1_heatmap.html")
print(f"   • Puntos en HeatMap: {len(heat_data)}")
print(f"   • Radio: 15, Blur: 10")

# Mostrar
mapa1

In [ ]:
# 5. MAPA 2: MARCADORES INDIVIDUALES
print("\n📍 CREANDO MAPA 2: MARCADORES INDIVIDUALES")

# Crear mapa base
mapa2 = folium.Map(location=[40, -110], zoom_start=6, tiles='OpenStreetMap')

# Función para icono de fuego
def icono_fuego(color):
    return folium.Icon(color=color, icon='fire', prefix='fa')

# Añadir marcadores (solo 100 para mejor rendimiento)
for _, row in df.head(100).iterrows():
    popup_text = f"""
    <b>🔥 Incendio</b><br>
    <b>Intensidad:</b> {row['intensidad']}<br>
    <b>Brightness:</b> {row['brightness']:.1f}<br>
    <b>Satélite:</b> {row['satellite']}
    """
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=250),
        icon=icono_fuego(row['color']),  # REQUISITO: icono fire con color según brightness
        tooltip=f"Intensidad: {row['intensidad']}"
    ).add_to(mapa2)

# Guardar
mapa2.save('mapa2_marcadores.html')
print("✅ Mapa 2 guardado: mapa2_marcadores.html")
print("   • Iconos: fire (FontAwesome)")
print("   • Colores: amarillo/naranja/rojo según brightness")
print("   • Popups con información")

# Mostrar
mapa2

In [ ]:
# 6. MAPA 3: MARKERCLUSTER
print("\n👥 CREANDO MAPA 3: MARKERCLUSTER")

# Crear mapa base
mapa3 = folium.Map(location=[40, -110], zoom_start=5, tiles='OpenStreetMap')

# Crear MarkerCluster (REQUISITO 3)
marker_cluster = MarkerCluster(
    name="Incendios Agrupados",
    options={'maxClusterRadius': 50, 'disableClusteringAtZoom': 12}
).add_to(mapa3)

# Añadir todos los marcadores al cluster
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=icono_fuego(row['color'])
    ).add_to(marker_cluster)

# Guardar
mapa3.save('mapa3_cluster.html')
print("✅ Mapa 3 guardado: mapa3_cluster.html")
print(f"   • Incendios agrupados: {len(df)}")
print(f"   • Radio del cluster: 50")

# Mostrar
mapa3

In [ ]:
# 7. MAPA 4: CONTROL POR SATÉLITE
print("\n🛰️ CREANDO MAPA 4: CONTROL POR SATÉLITE")

# Crear mapa base
mapa4 = folium.Map(location=[40, -110], zoom_start=6, tiles='OpenStreetMap')

# Crear FeatureGroups para cada satélite (REQUISITO 4)
aqua_group = folium.FeatureGroup(name='Aqua (Azul)', show=True)
terra_group = folium.FeatureGroup(name='Terra (Verde)', show=True)

# Añadir marcadores a los grupos correspondientes
for _, row in df.iterrows():
    if row['satellite'] == 'Aqua':
        grupo = aqua_group
        color = 'blue'
    else:
        grupo = terra_group
        color = 'green'
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.Icon(color=color, icon='satellite', prefix='fa')
    ).add_to(grupo)

# Añadir grupos al mapa
aqua_group.add_to(mapa4)
terra_group.add_to(mapa4)

# Añadir control de capas (REQUISITO)
folium.LayerControl(collapsed=False).add_to(mapa4)

# Guardar
mapa4.save('mapa4_satelite.html')
print("✅ Mapa 4 guardado: mapa4_satelite.html")
print("   • FeatureGroups por satélite")
print("   • Control de capas activado")
print("   • Colores: Azul (Aqua), Verde (Terra)")

# Mostrar
mapa4

In [ ]:
# 8. MAPA 5: CONTROL POR INTENSIDAD
print("\n🎨 CREANDO MAPA 5: CONTROL POR INTENSIDAD")

# Crear mapa base
mapa5 = folium.Map(location=[40, -110], zoom_start=6, tiles='OpenStreetMap')

# Crear FeatureGroups para cada intensidad (REQUISITO 5)
amarillo_group = folium.FeatureGroup(name='Baja (Amarillo)', show=True)
naranja_group = folium.FeatureGroup(name='Media (Naranja)', show=True)
rojo_group = folium.FeatureGroup(name='Alta (Rojo)', show=True)

# Añadir marcadores a los grupos correspondientes
for _, row in df.iterrows():
    if row['color'] == 'yellow':
        grupo = amarillo_group
    elif row['color'] == 'orange':
        grupo = naranja_group
    else:
        grupo = rojo_group
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=icono_fuego(row['color'])
    ).add_to(grupo)

# Añadir grupos al mapa
amarillo_group.add_to(mapa5)
naranja_group.add_to(mapa5)
rojo_group.add_to(mapa5)

# Añadir control de capas (REQUISITO)
folium.LayerControl(collapsed=False).add_to(mapa5)

# Guardar
mapa5.save('mapa5_intensidad.html')
print("✅ Mapa 5 guardado: mapa5_intensidad.html")
print("   • FeatureGroups por intensidad")
print("   • Control de capas activado")
print("   • Filtros: Amarillo/Naranja/Rojo")

# Mostrar
mapa5

In [ ]:
# 9. RESUMEN FINAL
print("\n" + "="*60)
print("📁 ARCHIVOS GENERADOS")
print("="*60)

archivos = [
    "mapa1_heatmap.html",
    "mapa2_marcadores.html",
    "mapa3_cluster.html",
    "mapa4_satelite.html",
    "mapa5_intensidad.html",
    "fires_map.csv"
]

for archivo in archivos:
    print(f"📄 {archivo}")

print("\n" + "="*60)
print("🎯 REQUISITOS CUMPLIDOS")
print("="*60)

requisitos = [
    "1. HeatMap con folium.plugins.HeatMap ✓",
    "2. Marcadores con icono fire y colores según brightness ✓",
    "3. Agrupación con MarkerCluster ✓",
    "4. FeatureGroup por satélite + LayerControl ✓",
    "5. FeatureGroup por intensidad + LayerControl ✓"
]

for req in requisitos:
    print(f"✅ {req}")

print("\n🚀 PROYECTO LISTO PARA ENTREGA")